In [1]:
import os, sys
sys.path.append("..")

# Optimization of Pauli Poly

There are two measure to estimate performances of `PauliPoly`.

- Naive `PauliPoly` implementation.
- Qiskit `SparsePauliOp` and `PauliList`.
- 
The PauliPoly in the origial OptTrot library was a naive implementation written by python.
In here, we will optimize the routine as soon as possible using scipy and numba routines.

In [132]:
from opttrot.pauli import PauliElement
from opttrot.pauli import PauliPoly

import numpy as np
import matplotlib.pyplot as plt

In [161]:
from importlib import reload
import paulipoly
reload(paulipoly)
from paulipoly import PauliPoly as NewPauliPoly
import ten_con
reload(ten_con)
from ten_con import _mat_to_coef_mat, _coef_to_mat

In [140]:
from qiskit.quantum_info import Pauli, PauliList, SparsePauliOp

In [134]:
n = 3
plist1 = [
    PauliElement(3, 5, n, 3),
    PauliElement(3, 1, n, 3),
    PauliElement(0, 4, n, 3),
    PauliElement(4, 2, n, 3),
]
plist2 = [
    PauliElement(1, 1, n, 1),
    PauliElement(3, 0, n, 1),
    PauliElement(2, 2, n, 1),
    PauliElement(2, 5, n, 1),
    PauliElement(4, 4, n, 1),
    PauliElement(3, 5, n, 1),
    PauliElement(3, 1, n, 1),
]

In [164]:
qubits = 10
ndim = int(2**qubits)
A = np.random.rand(ndim,ndim)
B = np.random.rand(ndim,ndim)
A[A<0.3] = 0.
B[B<0.9] = 0.
test_ppoly = PauliPoly.from_coef_mat(A)

In [165]:
plist = list(test_ppoly._terms.values())
coef_mat = test_ppoly.coef_matrix
mat = test_ppoly.matrix

## Coef matrix to PauliPoly

In [166]:
%%timeit
PauliPoly.from_coef_mat(coef_mat)

1min 37s ± 479 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [167]:
%%timeit
NewPauliPoly.from_coef_mat(coef_mat)

8.18 s ± 293 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Hermit matrix to PauliPoly

In [146]:
%%timeit
PauliPoly.from_matrix(mat)

134 ms ± 3.32 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [168]:
%%timeit
NewPauliPoly.from_matrix(mat)

11.5 s ± 131 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [170]:
%%timeit
SparsePauliOp.from_operator(mat)

252 ms ± 1.48 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Addition

In [162]:
npp001 = NewPauliPoly.from_coef_mat(A)
npp002 = NewPauliPoly.from_coef_mat(B)

**Comparsion: Matrix and set methods**

Qubits = 5

- Matrix: 58.3 ms ± 2.4 ms
- Set: 13.2 ms ± 111 µs

Qubits = 8

- Matrix: 3.68 s ± 67 ms per loop
- Set: 2 s ± 74.5 ms

In [163]:
%%timeit
npp001+npp002

3.68 s ± 67 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [160]:
%%timeit
npp001+npp002

2 s ± 74.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [152]:
pp001 = PauliPoly.from_coef_mat(A)
pp002 = PauliPoly.from_coef_mat(B)

In [153]:
%%timeit
pp001+pp002

144 ms ± 1.69 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# NewPauliPoly

In [ ]:
np.bitwise_xor

In [71]:
from scipy.sparse import coo_matrix
from scipy.sparse import csr_matrix

In [83]:
sparse_mat_coo_a = coo_matrix(A)
sparse_mat_coo_b = coo_matrix(B)
sparse_mat_coo_c = coo_matrix(A+B)


In [79]:
sparse_mat_coo_a.row

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9, 10, 10, 10, 10

In [86]:
(sparse_mat_coo_a + sparse_mat_coo_b).tocoo().row

array([ 0,  0,  0, ..., 31, 31, 31])

In [88]:
sparse_mat_coo_c.row

array([ 0,  0,  0, ..., 31, 31, 31])

In [7]:
mat = np.random.rand(4,5)

In [19]:
mat

array([[0.98396114, 0.11431422, 0.64416225, 0.83829608, 0.76007671],
       [0.78787825, 0.88660503, 0.4304859 , 0.63318161, 0.97439793],
       [0.1481726 , 0.4065548 , 0.3605898 , 0.65183366, 0.29498394],
       [0.69900726, 0.11389584, 0.3069482 , 0.1730905 , 0.60415335]])

In [22]:
it = np.nditer(mat, flags=["multi_index"])
for x in it:
    print(x, it.multi_index)

0.9839611397380185 (0, 0)
0.11431421595819469 (0, 1)
0.6441622493972661 (0, 2)
0.8382960792968918 (0, 3)
0.7600767139297588 (0, 4)
0.7878782457318573 (1, 0)
0.8866050342664579 (1, 1)
0.4304859034202665 (1, 2)
0.6331816148866182 (1, 3)
0.9743979294313301 (1, 4)
0.14817259918107117 (2, 0)
0.40655480478191863 (2, 1)
0.36058980389061357 (2, 2)
0.6518336648043467 (2, 3)
0.2949839434777849 (2, 4)
0.6990072574673085 (3, 0)
0.11389583579234219 (3, 1)
0.30694820053487315 (3, 2)
0.17309049971832735 (3, 3)
0.6041533460284433 (3, 4)


np.random

In [92]:
parr_1 = np.array(plist1) 
parr_2 = np.array(plist2) 

In [97]:
common_001  = np.intersect1d(parr_2, parr_1)
common_002  = np.intersect1d(parr_1, parr_2)

In [98]:
common_001

array([PauliElement(n=3, weight=3.000000+(0.000000)j, IXY),
       PauliElement(n=3, weight=3.000000+(0.000000)j, ZXY)], dtype=object)

In [99]:
common_002

array([PauliElement(n=3, weight=1.000000+(0.000000)j, IXY),
       PauliElement(n=3, weight=1.000000+(0.000000)j, ZXY)], dtype=object)

In [101]:
common = common_001 + common_002

In [102]:
sep = np.setxor1d(parr_1, parr_2)

In [105]:
arr = np.concatenate([common, sep])
arr.sort()
arr

array([PauliElement(n=3, weight=3.000000+(0.000000)j, ZII),
       PauliElement(n=3, weight=1.000000+(0.000000)j, IIY),
       PauliElement(n=3, weight=1.000000+(0.000000)j, IYI),
       PauliElement(n=3, weight=1.000000+(0.000000)j, ZXZ),
       PauliElement(n=3, weight=1.000000+(0.000000)j, IXX),
       PauliElement(n=3, weight=4.000000+(0.000000)j, IXY),
       PauliElement(n=3, weight=4.000000+(0.000000)j, ZXY),
       PauliElement(n=3, weight=3.000000+(0.000000)j, XZI),
       PauliElement(n=3, weight=1.000000+(0.000000)j, YII)], dtype=object)

In [ ]:
np.sort()